# Utilities
> Useful function and utilities

In [1]:
# default_exp utils

## Image

### Cleaning up error image

In [6]:
# export
from pathlib import Path
from tqdm.notebook import tqdm
import os
import logging
from typing import Callable, List
from PIL import Image

In some cases, we have error image that will interrupt model trainging. We can use ```clean_error_img``` to clean the image folder

In [7]:
# export
def check_img(
    img: Path,
    formats: List[str] = [".jpg", ".jpeg", ".png", ".bmp"],
) -> None:
    """
    Check on a single image,
    If it's quality is troublesome
        we unlink/ditch the image
    """
    img = Path(img)
    # check if this path is an image
    if img.suffix.lower().split("?")[0] not in formats:
        return

    try:
        # try to open that image
        _ = Image.open(img).load()
    except Exception as e:
        if img.exists():
            img.unlink()
            logging.warning(f"removed error img: {img}")


def clean_error_img(
    path: Path,
    progress: bool = True,
) -> None:
    """
    - path: an image directory
    - progress: do we print out progress bar or not
        default True
    """
    path = Path(path)

    # check directory existence
    if path.exists() == False:
        raise FileExistsError(
            f"""path does not exists on:{path}, 
    make sure there is a directory "{path.name}".
    under directory "{path.parent}"
    """)

    # create iterator, probably with progress bar
    iterator = tqdm(list(path.iterdir()), leave=False)\
        if progress else path.iterdir()

    for obj in iterator:
        if obj.is_dir():
            # use recursion to clean the sub folder
            clean_error_img(obj, progress=progress)
        else:
            # cheking on a single image
            check_img(obj)

## Testing

### test image cleaner

In [8]:
# hide
def testing_image_cleaner(func: Callable) -> None:
    path = Path("./test_remove")
    os.system(f"rm -rf {path}")
    path.mkdir()
    with open(path/f"file_1.JPEG", "w") as f:
        f.write("fake image")
        
    with open(path/f"file_1.jpg", "w") as f:
        f.write("fake image")
        
    with open(path/f"file_1.txt", "w") as f:
        f.write("not image")
        
    with open(path/f"file_2.png", "w") as f:
        f.write("fake image")
    
    sub1 = path/"sub1"
    sub2 = path/"sub2"
    sub1.mkdir()
    sub2.mkdir()
    sub3 = sub2/"sub3"
    sub3.mkdir()
    
    with open(sub1/"file3.BMP", "w") as f:
        f.write("fake image")
        
    with open(sub1/"file3 haha.txt", "w") as f:
        f.write("not image")
        
    with open(sub1/"😱file3 haha.jpg", "w") as f:
        f.write("fake image")
        
    with open(sub2/"file3.jpg", "w") as f:
        f.write("fake image")
    
    with open(sub2/"file4.jpeg", "w") as f:
        f.write("fake image")
        
    file5 = sub3/"file_5.jpeg"
    with open(file5, "w") as f:
        f.write("fake image")
        
    func(path)
    for p in [path, sub1, sub2, sub3]:
        print(list(p.iterdir()))
    os.system(f"rm -rf {path}")

In [9]:
# hide
testing_image_cleaner(clean_error_img)

[PosixPath('test_remove/file_1.txt'), PosixPath('test_remove/sub1'), PosixPath('test_remove/sub2')]
[PosixPath('test_remove/sub1/file3 haha.txt')]
[PosixPath('test_remove/sub2/sub3')]
[]


## Magic function ```hush```
> Run things quietly...

In [140]:
# export
from IPython.core.magic import register_cell_magic
from IPython.display import Javascript
from ipywidgets import Output, VBox, Button, HTML
import logging
from uuid import uuid4

This is a magical cell function, so remember to use the ```%%```

Now we have 2 versions of hush, the 1st on is the backup one, that using ipywidget as event trigger

In [99]:
@register_cell_magic
def hush(line, cell):
    """
    A magic cell function to collapse the print out
    %%hush
    how_loud = 100
    how_verbose = "very"
    do_some_python_thing(how_loud, how_verbose)
    """
    # the current output widget
    out = Output()
    output_box = VBox([out])
    # default setting is to hide the print out
    output_box.layout.display = "none"
    # the toggling button
    show_btn = Button(description="Show output")

    def toggle_output(o):
        # show output, change the button to hide
        if output_box.layout.display == "none":
            output_box.layout.display = "block"
            show_btn.description = "Hide output"
        # hide output, change the button to show
        else:
            output_box.layout.display = "none"
            show_btn.description = "Show output"

    # assign toggle to event
    show_btn.on_click(toggle_output)

    # A control panel containing a button
    # and the output box
    total_control = VBox([show_btn, output_box])
    display(total_control)

    with out:
        ishell = get_ipython()
        # excute the code in cell
        result = ishell.run_cell(
            cell, silent=False)

    # we still want the error to be proclaimed loudly
    if result.error_in_exec:
        logging.error(f"'{result.error_in_exec}' happened, breaking silence now")
        result.raise_error()

But this version of ```toggle action``` will be stuck by the ongoing interactive

If the process is working on some thing, you can't toggle until the end of the execution.

The further improvement will be move the toggle entirely to JavaScript, hence the second version

### Hush event in JS
> As not going through any amount of python backend after run

In [160]:
# export
@register_cell_magic
def hush(line, cell):
    """
    A magic cell function to collapse the print out
    %%hush
    how_loud = 100
    how_verbose = "very"
    do_some_python_thing(how_loud, how_verbose)
    """
    # the current output widget
    out = Output()
    output_box = VBox([out])
    
    # create uuid for DOM identifying
    uuid = str(uuid4())

    # default setting is to hide the print out
    output_box.layout.display = "none"
    # the toggling button
    show_btn = Button(description="toggle output")
    show_btn.add_class(f"hush_toggle_{uuid}")
    output_box.add_class(f"hush_output_{uuid}")
    
    
    # A control panel containing a button
    # and the output box
    total_control = VBox([show_btn, output_box])
    display(total_control)
    
    # assign JS listener
    display(Javascript(f"""
    console.info("loading toggle event: {uuid}")
    const toggle_hush = (e) =>\u007b
        var op = document.querySelector(".hush_output_{uuid}");
        if(op.style.display=="none")\u007b
             op.style.display="block"

        \u007d else \u007b
            op.style.display="none"
        \u007d

    \u007d
    document.querySelector('.hush_toggle_{uuid}').onclick=toggle_hush
    """))

    with out:
        ishell = get_ipython()
        # excute the code in cell
        result = ishell.run_cell(
            cell, silent=False)

    # we still want the error to be proclaimed loudly
    if result.error_in_exec:
        logging.error(f"'{result.error_in_exec}' happened, breaking silence now")
        result.raise_error()

### Try hushing various kinds of info

* html display
* print
* logging

In [161]:
%%hush
import pandas as pd
import logging
logging.getLogger().setLevel('DEBUG')
a = 1
for i in range(1000):
    print(i, end="\t")
    
logging.info("a lots of text, VERBOSITY!!"*100)

# a big dataframe
display(pd.DataFrame({"a":[1,2]*50}))

# a big output
pd.DataFrame({"b":range(100)})

<IPython.core.display.Javascript object>

### Capable of open/close verbosity during the main process

In [162]:
%%hush
from tqdm.notebook import tqdm
from time import sleep
for i in tqdm(range(10)):
    sleep(1)

<IPython.core.display.Javascript object>

### But we still want the error to be loud
> As such information should interrupt the user

In [163]:
%%hush
for i in range(20):
    print(f"some logging:\t{i}!")
raise ValueError("but some error, because life!")

<IPython.core.display.Javascript object>

ERROR:root:'but some error, because life!' happened, breaking silence now


ValueError: but some error, because life!